In [1]:
#================================================
# EXPLORAÇÃO INICIAL DA TABELA "movies"
#================================================

import duckdb, pandas as pd
from pathlib import Path

# cria ou liga se já existir:
con = duckdb.connect("movielens100K.duckdb")

In [2]:
con.sql("DESCRIBE movies").df()


,column_name,column_type,null,key,default,extra
0,movieId,BIGINT,YES,None,None,None
1,title,VARCHAR,YES,None,None,None
2,genres,VARCHAR,YES,None,None,None


### Comentário
 - "movieID": BIGINT 64 bits value;
 - "title": movie name in VARCHAR → texto
 - "genres": Genres name in VARCHAR → texto

 - Coluna "null": indica se a coluna pode conter valores nulos (NULL).
 - Neste caso pode.
 
 - Coluna "key": indica se coluna é chave primária (PRIMARY KEY).
 - Não faz.

 - Coluna "default": indica se mostra o valor por defeito (DEFAULT).
 - Não tem

 - Coluna "extra": mostra informações adicionais sobre a coluna, como: auto_increment ou generated.
 - Neste caso não tem

In [3]:
con.sql("PRAGMA table_info('movies')").df()


,cid,name,type,notnull,dflt_value,pk
0,0,movieId,BIGINT,False,None,False
1,1,title,VARCHAR,False,None,False
2,2,genres,VARCHAR,False,None,False


#### Comentário

 - "movieID": BIGINT 64 bits value;
 - "title": movie name in VARCHAR → texto
 - "genres": Genres name in VARCHAR → texto

 - A coluna "notnull boolean" indica 3 valores zeros.
 - Assim esta tabela aceita nulos ou NULL.
 - Como df foi carregado com read_csv_auto, DuckDB não aplica restrições de NOT NULL

 - Coluna flt_value mostra valor por defeito atribuído caso não seja especificado outro valor quando fazes um INSERT.
 - Nos datasets importados com read_csv_auto (como o MovieLens), esta coluna vai quase sempre aparecer como NULL

 - pk boolean indica se aquela coluna faz parte da chave primária da tabela
 - Não faz parte

In [4]:
#Ver primeiras 10 linhas
con.sql("SELECT * FROM movies LIMIT 10").df()


,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy
5,6,Heat (1995),Action|Crime|Thriller
6,7,Sabrina (1995),Comedy|Romance
7,8,Tom and Huck (1995),Adventure|Children
8,9,Sudden Death (1995),Action
9,10,GoldenEye (1995),Action|Adventure|Thriller


In [5]:
#Contar o número total de linhas
con.sql("SELECT COUNT(*) AS total_movies FROM movies").df()


,total_movies
0,9742


#### CONCLUSÃO:
 - Esta tabela serve para identificar os filmes presentes na presente database.
 - Identifica o ID de cada filme e o genero associado. 
 - Não tem chaves primárias
 - Temos no total 9742 filmes.
 - Cada filme pode ter mais do que um estilo.

In [6]:
#Contar o número de missing values
con.sql("""
SELECT
    COUNT(*) - COUNT(movieId) AS missing_movieId,
    COUNT(*) - COUNT(title)   AS missing_title,
    COUNT(*) - COUNT(genres)  AS missing_genres
FROM movies
""").df()


,missing_movieId,missing_title,missing_genres
0,0,0,0


#### CONCLUSÃO:
 - Não tem missing values.

In [7]:
#Ver distribuição de valores
con.sql("""
SELECT genres, COUNT(*) AS n
FROM movies
GROUP BY genres
ORDER BY n DESC
LIMIT 20
""").df()


,genres,n
0,Drama,1053
1,Comedy,946
2,Comedy|Drama,435
3,Comedy|Romance,363
4,Drama|Romance,349
5,Documentary,339
6,Comedy|Drama|Romance,276
7,Drama|Thriller,168
8,Horror,167
9,Horror|Thriller,135


Verifica-se que podem haver misturas de estilos que podem tornar difícil a quantificação dos estilos;
Exemplo Drama, Comedy| Drama e etc o que pode dificultar a contabilização

In [8]:
#Contar quantos filmes temos por género sabendo que o campo genres no MovieLens tem vários géneros por filme, separados por barras (|)
con.sql("""
SELECT
    genre,
    COUNT(*) AS total_filmes
FROM (
    SELECT unnest(string_split(genres, '|')) AS genre
    FROM movies
)
GROUP BY genre
ORDER BY total_filmes DESC
""").df()


,genre,total_filmes
0,Drama,4361
1,Comedy,3756
2,Thriller,1894
3,Action,1828
4,Romance,1596
5,Adventure,1263
6,Crime,1199
7,Sci-Fi,980
8,Horror,978
9,Fantasy,779


In [9]:
#Ver quantos géneros em média tem cada filme
con.sql("""
SELECT
    AVG(array_length(string_split(genres, '|'))) AS media_generos_por_filme
FROM movies
""").df()


,media_generos_por_filme
0,2.266886


In [10]:
#Ver filmes com o maior número de géneros
con.sql("""
SELECT
    title,
    genres,
    array_length(string_split(genres, '|')) AS n_generos
FROM movies
ORDER BY n_generos DESC, title
LIMIT 10
""").df()


,title,genres,n_generos
0,Rubber (2010),Action|Adventure|Comedy|Crime|Drama|Film-Noir|...,10
1,Patlabor: The Movie (Kidô keisatsu patorebâ: T...,Action|Animation|Crime|Drama|Film-Noir|Mystery...,8
2,Aelita: The Queen of Mars (Aelita) (1924),Action|Adventure|Drama|Fantasy|Romance|Sci-Fi|...,7
3,Aqua Teen Hunger Force Colon Movie Film for Th...,Action|Adventure|Animation|Comedy|Fantasy|Myst...,7
4,Enchanted (2007),Adventure|Animation|Children|Comedy|Fantasy|Mu...,7
5,Inception (2010),Action|Crime|Drama|Mystery|Sci-Fi|Thriller|IMAX,7
6,Interstate 60 (2002),Adventure|Comedy|Drama|Fantasy|Mystery|Sci-Fi|...,7
7,Mars Needs Moms (2011),Action|Adventure|Animation|Children|Comedy|Sci...,7
8,Mulan (1998),Adventure|Animation|Children|Comedy|Drama|Musi...,7
9,Osmosis Jones (2001),Action|Animation|Comedy|Crime|Drama|Romance|Th...,7


Aconselho verem o trailer do filme que tem mais estilos... é qualquer coisa: 

#### Fechar a ligação

In [11]:
con.close()
print("Ligação fechada.")

Ligação fechada.
